## FFE with geopandas

This is cut-n-paste of original approach for future reference.

In [1]:
import datetime
import glob
import math
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
import networkx as nx
from shapely.geometry import Point
import imageio

pd.options.mode.chained_assignment = None  # default='warn'


In [2]:
path = "/geodata"

In [3]:

def load_data(file_name, minx, miny, maxx, maxy):
    # crop data
    bbox = box(minx, miny, maxx, maxy)
    # building point dataset
    gdf_buildings = gpd.read_file(os.path.join(path, file_name), bbox=bbox)
    # gdf_buildings.IgnProb_bl = 0.02
    # xmin,ymin,xmax,ymax = gdf_buildings.total_bounds
    return gdf_buildings

In [4]:
# set up & load input data
# gdf = load_data("buildings_raw_pts.shp", 1748570, 5426959, 1748841, 5427115)
# gdf_polygon = load_data("shapes_1200.shp"), 178570, 5426959, 1748841, 5427115)

## No cropping, there are only 1728 assets

gdf_polygon = gpd.read_file(os.path.join(path, "shapes_1200.shp"))
                            
gdf_polygon["area"] = gdf_polygon['geometry'].area  # m2
gdf = gdf_polygon.copy()
gdf['geometry'] = gdf['geometry'].centroid
gdf['X'] = gdf.centroid.x
gdf['Y'] = gdf.centroid.y
gdf['d_short'] = gdf_polygon.exterior.distance(gdf)
gdf['d_long'] = gdf['area'] / gdf['d_short']


## Costly function

Observations - **using pd.merge** note that only one core is doing the work.

Some potential to get spark + geomesa in action here??

In [5]:

def build_edge_list(geodataframe, maximum_distance, polygon_file):
    # create arrays for different id combination
    n = np.arange(0, len(geodataframe))
    target = [n] * len(geodataframe)
    target = np.hstack(target)
    source = np.repeat(n, len(geodataframe))

    # put arrays in dataframe
    df = pd.DataFrame()
    df['source_id'] = source
    df['target_id'] = target
    # merge source attributes with source index
    geo_df = geodataframe.copy()
    geo_df['id'] = geo_df.index
    
    # create source / target gdf from gdf.columns of interest
    geo_df = geo_df[['id', 'TARGET_FID', 'X', 'Y', 'geometry', 'IgnProb_bl']]
    geo_df_TRG = geo_df.copy()
    geo_df_TRG.columns = ['target_' + str(col) for col in geo_df_TRG.columns]
    geo_df_SRC = geo_df.copy()
    geo_df_SRC.columns = ['source_' + str(col) for col in geo_df_SRC.columns]
    
    # merge data
    merged_data = pd.merge(df, geo_df_SRC, left_on='source_id', right_on='source_id', how='outer')
    merged_data = pd.merge(merged_data, geo_df_TRG, left_on='target_id', right_on='target_id', how='outer')
    merged_data.rename(columns={'source_id': 'source', 'target_id': 'target'}, inplace=True)
    
    # calculate distance for each source / target pair
    # create a df from polygon shape to get accurate distance
    # print(list(polygon_file))
    polygon = polygon_file[['TARGET_FID', 'geometry']]
    # print(list(polygon))
    source_poly = merged_data[['source_TARGET_FID']]
    target_poly = merged_data[['target_TARGET_FID']]
    # print(list(source_poly))
    src_poly = pd.merge(source_poly, polygon, left_on='source_TARGET_FID', right_on='TARGET_FID', how='left')
    trg_poly = pd.merge(target_poly, polygon, left_on='target_TARGET_FID', right_on='TARGET_FID', how='left')
    src_poly_gdf = gpd.GeoDataFrame(src_poly, geometry='geometry')
    trg_poly_gdf = gpd.GeoDataFrame(trg_poly, geometry='geometry')
    distance_series = src_poly_gdf.distance(trg_poly_gdf)
    # print(distance_series)

    # insert distance in merged data column
    merged_data['v1'] = merged_data.source_X - merged_data.target_X
    merged_data['v2'] = merged_data.source_Y - merged_data.target_Y
    # merged_data['euc_distance'] = np.hypot(merged_data.v1, merged_data.v2)
    merged_data['euc_distance'] = distance_series
    # remove when distance "illegal"
    valid_distance = merged_data['euc_distance'] < maximum_distance
    not_same_node = merged_data['euc_distance'] != 0
    data = merged_data[valid_distance & not_same_node]
    # calculate azimuth
    data['azimuth'] = np.degrees(np.arctan2(merged_data['v2'], merged_data['v1']))
    data['bearing'] = (data.azimuth + 360) % 360
    return data


In [6]:
%%time
## create edge list and network
edges = build_edge_list(gdf, 450, gdf_polygon)


CPU times: user 1min 16s, sys: 3.08 s, total: 1min 19s
Wall time: 1min 19s


In [8]:
edges

,source,target,source_TARGET_FID,source_X,source_Y,source_geometry,source_IgnProb_bl,target_TARGET_FID,target_X,target_Y,target_geometry,target_IgnProb_bl,v1,v2,euc_distance,azimuth,bearing
1,1,0,5477,1.745653e+06,5.427740e+06,POINT (1745652.533 5427740.364),0.000352,58152,1.745709e+06,5.428056e+06,POINT (1745708.763 5428056.083),0.000507,-56.229879,-315.719776,305.695300,-100.098526,259.901474
3,3,0,5478,1.745684e+06,5.427602e+06,POINT (1745684.194 5427602.138),0.000352,58152,1.745709e+06,5.428056e+06,POINT (1745708.763 5428056.083),0.000507,-24.568353,-453.944949,437.603191,-93.097933,266.902067
4,4,0,58153,1.745710e+06,5.428035e+06,POINT (1745710.337 5428035.235),0.000507,58152,1.745709e+06,5.428056e+06,POINT (1745708.763 5428056.083),0.000507,1.574096,-20.847924,3.609837,-85.682146,274.317854
5,5,0,5479,1.745683e+06,5.427617e+06,POINT (1745683.392 5427616.885),0.000352,58152,1.745709e+06,5.428056e+06,POINT (1745708.763 5428056.083),0.000507,-25.370903,-439.198446,423.188670,-93.306095,266.693905
7,7,0,5480,1.745686e+06,5.427642e+06,POINT (1745686.474 5427642.381),0.000352,58152,1.745709e+06,5.428056e+06,POINT (1745708.763 5428056.083),0.000507,-22.288260,-413.702686,400.285922,-93.083833,266.916167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3090555,1749,1757,5471,1.745713e+06,5.427671e+06,POINT (1745713.040 5427671.377),0.000352,5476,1.745658e+06,5.427726e+06,POINT (1745657.872 5427726.455),0.000352,55.168700,-55.077425,71.110063,-44.952564,315.047436
3090557,1751,1757,5472,1.745714e+06,5.427644e+06,POINT (1745713.866 5427644.450),0.000352,5476,1.745658e+06,5.427726e+06,POINT (1745657.872 5427726.455),0.000352,55.993925,-82.005075,88.709266,-55.674329,304.325671
3090558,1752,1757,5473,1.745700e+06,5.427658e+06,POINT (1745700.370 5427658.334),0.000352,5476,1.745658e+06,5.427726e+06,POINT (1745657.872 5427726.455),0.000352,42.498440,-68.120868,71.046173,-58.041274,301.958726
3090560,1754,1757,5474,1.745695e+06,5.427695e+06,POINT (1745695.056 5427695.003),0.000352,5476,1.745658e+06,5.427726e+06,POINT (1745657.872 5427726.455),0.000352,37.183961,-31.451685,36.063110,-40.225881,319.774119


In [9]:
len(edges)

1153266

In [64]:
str(r.geometry.centroid)

'1    POINT (1745652.533 5427740.364)\ndtype: geometry'